In [159]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# Initialize Data Frame

In [160]:
testRaw = pd.read_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/test.csv')
testRaw.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl        NAmes      Feedr   
1         Lvl    AllPub    Corner       Gtl        NAmes       Norm   
2         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
3         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
4         HLS    AllPub    Inside       Gtl      StoneBr       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     1Story            5            6       1961   
1       Norm     1Fam     1Story            6            6       1958   
2       Norm     1Fam     2Story            5            5       1997   
3       Norm     1Fam     2Story            6            6       1998   
4       Norm   TwnhsE     1Story            8            5       1992   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          1961     Gable  CompShg     VinylSd     VinylSd       None   
1          1958       Hip  CompShg     Wd Sdng     Wd Sdng    BrkFace   
2          1998     Gable  CompShg     VinylSd     VinylSd       None   
3          1998     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4          1992     Gable  CompShg     HdBoard     HdBoard       None   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0         0.0        TA        TA     CBlock       TA       TA           No   
1       108.0        TA        TA     CBlock       TA       TA           No   
2         0.0        TA        TA      PConc       Gd       TA           No   
3        20.0        TA        TA      PConc       TA       TA           No   
4         0.0        Gd        TA      PConc       Gd       TA           No   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          Rec       468.0          LwQ       144.0      270.0        882.0   
1          ALQ       923.0          Unf         0.0      406.0       1329.0   
2          GLQ       791.0          Unf         0.0      137.0        928.0   
3          GLQ       602.0          Unf         0.0      324.0        926.0   
4          ALQ       263.0          Unf         0.0     1017.0       1280.0   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        TA          Y      SBrkr       896         0             0   
1    GasA        TA          Y      SBrkr      1329         0             0   
2    GasA        Gd          Y      SBrkr       928       701             0   
3    GasA        Ex          Y      SBrkr       926       678             0   
4    GasA        Ex          Y      SBrkr      1280         0             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0        896           0.0           0.0         1         0             2   
1       1329           0.0           0.0         1         1             3   
2       1629           0.0           0.0         2         1             3   
3       1604           0.0           0.0         2         1             3   
4       1280           0.0           0.0         2         0             2   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          TA             5        Typ           0         NaN   
1             1          Gd             6        Typ           0         NaN   
2             1          TA             6        Typ           1          TA   
3             1          Gd  

##  Find NAs

In [161]:
testRaw.isna().sum(axis=0)

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

## Action Plan for NA values

**Cleaning NAs is going to be a process**  
+ All variables with quality in the description use NA to denote there is no object to denote the quality of  
    - suggest replacing with text 'none' and using that as the default case in dummification   
    - build a masking variable
+ MSZoning has 4 NA values, no easily identifiable characteristics to interpolate, strongly consider dropping. Could be a very important variable.
+ LotFrontage Variable has 227 NA values, but no values equal to zero, find way to impute, see below for special treatment
+ Utilities has 2 NA values, no identifying charactersitics for this variable strongly consider dropping  
+ Exterior1st has 1 NA value, convert to 'Other' 
+ Exterior2nd has 1 NA value, convert to 'Other' 
+ MasVnrArea has 16 NA values, convert to None  
+ MasVnrQual has 15 NA values corresponding to the NAs in previous category, change to 0 (an acceptable value in this dataset)
+ BsmtFinSF1 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFinSF2 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtUnfSF has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFullBath has 2 NAs, but records say there is no basement, change to zero  
+ BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero 
+ KitchenAbvGr has 1 NA, but has a kitchen listed in the record, no way to interpolate, consider dropping
+ Functional has 2 NA values, no way to interpolate consider dropping
+ GarageCars, and GarageArea NA values will be dropped (see below anomoly description)
+ SaleType, has 1 NA value, change to 'Oth'

**Variables with NA values who's rows we should drop**  
+ MSZoning  
+ Utilities
+ KitchenQual
+ Functional




### Mask Variable for replacing NA values in columns where NA has meaning

In [162]:
naMask = ['Alley', 'Exterior1st','Exterior2nd', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish'\
         ,'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature' ]

### Additional Data Anomolies

**Mismatch in Basement Existence Data**

In [163]:
testRaw[testRaw['BsmtQual'].isna()].loc[:,['BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinType2', 'BsmtFinSF2','BsmtUnfSF', 'TotalBsmtSF' ]]\
.loc[testRaw['BsmtUnfSF']!=0]

BsmtQual BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1 BsmtFinType2  \
660      NaN      NaN          NaN          NaN         NaN          NaN   
757      NaN       Fa           No          Unf         0.0          Unf   
758      NaN       TA           No          Unf         0.0          Unf   

     BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  
660         NaN        NaN          NaN  
757         0.0      173.0        173.0  
758         0.0      356.0        356.0

These 3 records have basement quality listed as 'No Basment', the last two records however have values listed for basement features, can not interpolate basement quality from data recomend dropping records 757 and 758.
For record 660 Set basement square footage columns to 0, replacing NA values to NONE in previous step will take care of the rest.

**Missing Column, Kitchen**

In the data dictionary file, a column named 'Kitchen' is listed, however the datafile has a column named 'KitchenAbvGr' instead

**Garage Existence Data Mismatch**

In [164]:
testRaw[testRaw['GarageYrBlt'].isna()].loc[~testRaw['GarageType'].isna(), ['GarageType','GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'GarageYrBlt']]

GarageType GarageFinish  GarageCars  GarageArea GarageQual GarageCond  \
666      Detchd          NaN         1.0       360.0        NaN        NaN   
1116     Detchd          NaN         NaN         NaN        NaN        NaN   

      GarageYrBlt  
666           NaN  
1116          NaN

2 records with garage existence listed but limited values listed for the rest with no legal option for NaN, consider dropping records 666, 1116

# Clean Data

## Drop Problematic records by row index
**Previously documented 4 records with uniterpolatable illegal NA values**


In [165]:
print(testRaw.shape)
testClean = testRaw.drop([757, 758, 666, 1116])
testClean.shape

(1459, 80)


(1455, 80)

## Drop Rows with Ilegal NA values in Suspected High Importance Columns

In [166]:
dropImportant = ['MSZoning','Utilities','KitchenQual','Functional']

In [167]:
testClean = testClean.loc[~testClean['MSZoning'].isna() & ~testClean['Utilities'].isna() & ~testClean['KitchenQual'].isna() & ~testClean['Functional'].isna()]
testClean.shape

(1448, 80)

## Change NA value of certain columns where NA has meaning to None

In [168]:
for i in range(len(naMask)):
    testClean[naMask[i]].fillna('None', inplace = True)

## Change Specific NA values depending on Context  

**Exterior1st has 1 NA value, convert to 'Other'**

In [169]:
testClean.Exterior1st.fillna('Other', inplace = True)

**Exterior2nd has 1 NA value, convert to 'Other'**

In [170]:
testClean.Exterior2nd.fillna('Other', inplace = True)

**MasVnrType has 16 NA values, convert to None**

In [171]:
testClean.MasVnrType.fillna('None', inplace = True)

**MasVnrArea has 15 NA values corresponding to the NAs in previous category, change to 0**

In [172]:
testClean.MasVnrArea.fillna(0, inplace = True)

**TotalBsmtSF has 1 NA value, change to 0**

In [173]:
testClean.TotalBsmtSF.fillna(0, inplace = True)

**BsmtFinSF1 has 1 NA value, change to 0**

In [174]:
testClean.BsmtFinSF1.fillna(0, inplace = True)

**BsmtFinSF2 has 1 NA value, change to 0**

In [175]:
testClean.BsmtFinSF2.fillna(0, inplace = True)

**BsmtUnfSF has 1 NA value, change to 0**

In [176]:
testClean.BsmtUnfSF.fillna(0, inplace = True)

**BsmtFullBath has 2 NAs, but records say there is no basement, change to zero**

In [177]:
testClean.BsmtFullBath.fillna(0, inplace = True)

**BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero**

In [178]:
testClean.BsmtHalfBath.fillna(0, inplace = True)

**SaleType, has 1 NA value, change to 'Oth'**

In [179]:
testClean.SaleType.fillna('Oth', inplace = True)

#### LotFrontage computations

In [180]:
testClean.corr().LotFrontage.sort_values()

MSSubClass      -0.449830
OverallCond     -0.085660
BsmtHalfBath    -0.043196
3SsnPorch       -0.037513
LowQualFinSF    -0.036639
2ndFlrSF        -0.031631
YrSold          -0.024691
Id               0.008478
MoSold           0.012131
EnclosedPorch    0.012852
KitchenAbvGr     0.016513
HalfBath         0.020879
BsmtFinSF2       0.047218
MiscVal          0.069324
BsmtUnfSF        0.095910
YearRemodAdd     0.099859
ScreenPorch      0.115082
YearBuilt        0.124526
BsmtFullBath     0.126676
PoolArea         0.134987
WoodDeckSF       0.159475
FullBath         0.168790
OpenPorchSF      0.183533
OverallQual      0.197242
BsmtFinSF1       0.207475
BedroomAbvGr     0.211024
MasVnrArea       0.250880
Fireplaces       0.260497
TotalBsmtSF      0.324311
GarageCars       0.340458
TotRmsAbvGrd     0.350077
GrLivArea        0.362405
GarageArea       0.379947
1stFlrSF         0.462818
LotArea          0.641144
LotFrontage      1.000000
Name: LotFrontage, dtype: float64

**LotArea has the highest positive correlation, and MSSubClass has the highest negative correlation**
**Perhaps if we group by Subclass and regress with LotArea we can get good values for LotFrontage**

In [181]:
A = testClean.loc[testClean['LotFrontage'].notna(),['MSSubClass', 'LotFrontage', 'LotArea']]

In [182]:
#from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(A[['LotArea']], A[['LotFrontage']])
RSquared = lm.score(A[['LotArea']], A[['LotFrontage']])
Coefficient = lm.coef_
print(f'This model has a score of {RSquared} and a coefficient of {Coefficient}')

This model has a score of 0.4110655340962701 and a coefficient of [[0.00321682]]


**That is bad, let's try grouping by MSSubClass**

In [183]:
AGroup = A.groupby('MSSubClass')
for name,group in AGroup:
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = lm.score(group[['LotArea']], group[['LotFrontage']])
    Coefficient = lm.coef_
    print(f'This model for MSSubClass = {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')


This model for MSSubClass = 20 has a score of 0.19099812446932796 and a coefficient of [[0.0017633]]
    There are 455 records in this group
This model for MSSubClass = 30 has a score of 0.3685575243306103 and a coefficient of [[0.00238457]]
    There are 62 records in this group
This model for MSSubClass = 40 has a score of 1.0 and a coefficient of [[0.00219573]]
    There are 2 records in this group
This model for MSSubClass = 45 has a score of 0.7935575128584776 and a coefficient of [[0.00544202]]
    There are 6 records in this group
This model for MSSubClass = 50 has a score of 0.3325083920536953 and a coefficient of [[0.00214506]]
    There are 131 records in this group
This model for MSSubClass = 60 has a score of 0.12627370659093506 and a coefficient of [[0.00254355]]
    There are 211 records in this group
This model for MSSubClass = 70 has a score of 0.5140896324306916 and a coefficient of [[0.00300555]]
    There are 60 records in this group
This model for MSSubClass = 75 ha

**Quick Idea, try grouping by subclass and lotshape

In [184]:
B = testClean.loc[testClean['LotFrontage'].notna(),['MSSubClass', 'LotShape', 'LotFrontage', 'LotArea']]

In [185]:
BGroup = B.groupby(['LotShape', 'MSSubClass'])
for name,group in BGroup: 
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = lm.score(group[['LotArea']], group[['LotFrontage']])
    Coefficient = lm.coef_
    print(f'This model for LotShape, MSSubClass {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')

This model for LotShape, MSSubClass ('IR1', 20) has a score of 0.01997332039363031 and a coefficient of [[0.0004845]]
    There are 133 records in this group
This model for LotShape, MSSubClass ('IR1', 30) has a score of 1.0 and a coefficient of [[0.0026441]]
    There are 2 records in this group
This model for LotShape, MSSubClass ('IR1', 50) has a score of 0.04886982570131415 and a coefficient of [[0.00120662]]
    There are 7 records in this group
This model for LotShape, MSSubClass ('IR1', 60) has a score of 0.12634836389127224 and a coefficient of [[0.00275212]]
    There are 104 records in this group
This model for LotShape, MSSubClass ('IR1', 70) has a score of 0.11231193776683412 and a coefficient of [[0.00168477]]
    There are 5 records in this group
This model for LotShape, MSSubClass ('IR1', 80) has a score of 0.10185133528055745 and a coefficient of [[0.00188946]]
    There are 21 records in this group
This model for LotShape, MSSubClass ('IR1', 85) has a score of 0.229296

C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn

This model for LotShape, MSSubClass ('Reg', 75) has a score of 0.012114305453288177 and a coefficient of [[0.0002231]]
    There are 6 records in this group
This model for LotShape, MSSubClass ('Reg', 80) has a score of 0.2570455005519471 and a coefficient of [[0.00407744]]
    There are 25 records in this group
This model for LotShape, MSSubClass ('Reg', 85) has a score of 0.2744647810185942 and a coefficient of [[0.00499288]]
    There are 15 records in this group
This model for LotShape, MSSubClass ('Reg', 90) has a score of 0.11847959846543432 and a coefficient of [[0.00243286]]
    There are 40 records in this group
This model for LotShape, MSSubClass ('Reg', 120) has a score of 0.6365285927030471 and a coefficient of [[0.00452798]]
    There are 34 records in this group
This model for LotShape, MSSubClass ('Reg', 160) has a score of 0.6472480484930114 and a coefficient of [[0.00523689]]
    There are 59 records in this group
This model for LotShape, MSSubClass ('Reg', 180) has a 

**Hrmm...rSquared for regualr lot sizes seem to be higher, let's single those out**

In [186]:
C = testClean.loc[(testClean['LotFrontage'].notna()) & (testClean['LotShape'] == 'Reg'),['MSSubClass', 'LotShape', 'LotFrontage', 'LotArea']]
C

MSSubClass LotShape  LotFrontage  LotArea
0             20      Reg         80.0    11622
8             20      Reg         85.0    10176
9             20      Reg         70.0     8400
11           160      Reg         21.0     1680
12           160      Reg         21.0     1680
...          ...      ...          ...      ...
1454         160      Reg         21.0     1936
1455         160      Reg         21.0     1894
1456          20      Reg        160.0    20000
1457          85      Reg         62.0    10441
1458          60      Reg         74.0     9627

[868 rows x 4 columns]

In [187]:
CGroup = C.groupby('MSSubClass')
CfitList = []
for name,group in CGroup: 
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = round(lm.score(group[['LotArea']], group[['LotFrontage']]),5)
    Coefficient = round(lm.coef_[0][0],5)
    print(f'This model for MSSubClass = {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')
    CfitList.append([name, RSquared, Coefficient])
rSum = 0
for i in CfitList:
    rSum = rSum + i[1]
print(f'The average R Squared for this model is {rSum/len(CfitList)}')

This model for MSSubClass = 20 has a score of 0.58545 and a coefficient of 0.00383
    There are 315 records in this group
This model for MSSubClass = 30 has a score of 0.39887 and a coefficient of 0.00249
    There are 59 records in this group
This model for MSSubClass = 40 has a score of 1.0 and a coefficient of 0.0022
    There are 2 records in this group
This model for MSSubClass = 45 has a score of 0.79356 and a coefficient of 0.00544
    There are 6 records in this group
This model for MSSubClass = 50 has a score of 0.38847 and a coefficient of 0.00259
    There are 123 records in this group
This model for MSSubClass = 60 has a score of 0.51412 and a coefficient of 0.00496
    There are 98 records in this group
This model for MSSubClass = 70 has a score of 0.54325 and a coefficient of 0.00304
    There are 55 records in this group
This model for MSSubClass = 75 has a score of 0.01211 and a coefficient of 0.00022
    There are 6 records in this group
This model for MSSubClass = 80

**Whats the breakdown of lotShape and MSSubClass for the NA values in LotFrontage**

In [188]:
D = testClean.loc[testClean['LotFrontage'].isna(),['MSSubClass', 'LotShape']]
Dmelt = pd.melt(D)
Dmelt.groupby(['variable', 'value']).size()

variable    value
LotShape    IR1      152
            IR2       13
            IR3        2
            Reg       58
MSSubClass  20        85
            30         7
            50         9
            60        64
            70         5
            75         1
            80        13
            85        10
            90        12
            120       12
            150        1
            160        4
            180        1
            190        1
dtype: int64

**These groupbys are going nowhere and I'm not find any consistent strong correlations based on other variables besides LotArea**

**#@!$ it, I'm just multiplying the average ratio of LotArea and LotFrontage to impute the value of LotArea**

In [189]:
E = testClean.loc[testClean['LotFrontage'].notna(),['LotArea', 'LotFrontage']]
fRatio = E.agg('mean')[1]/ E.mean()[0]

In [190]:
testClean.loc[testClean['LotFrontage'].isna(),'LotFrontage'] = testClean['LotArea'] * fRatio

## Check to see if we elimanted all NA values

In [191]:
testClean.isna().sum(axis=0)

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr

## A Thing of Beauty Emerges, no NA values, save to new csv

In [192]:
testClean.to_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/testZeroNAs.csv')

# Repeat Process for Train Dataset

## Initialize Dataset

In [193]:
trainRaw = pd.read_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/train.csv')
trainRaw.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5       2003   
1       Norm     1Fam     1Story            6            8       1976   
2       Norm     1Fam     2Story            7            5       2001   
3       Norm     1Fam     2Story            7            5       1915   
4       Norm     1Fam     2Story            8            5       2000   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1          1976     Gable  CompShg     MetalSd     MetalSd       None   
2          2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3          1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4          2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0         NaN   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1          Gd            

## Eliminate NA values

**We know certain columns' NA has meaning, so we are going to change those NA values to 'None'**

In [194]:
trainClean = trainRaw
for i in range(len(naMask)):
    trainClean[naMask[i]].fillna('None', inplace = True)
trainClean.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave  None      Reg   
1   2          20       RL         80.0     9600   Pave  None      Reg   
2   3          60       RL         68.0    11250   Pave  None      IR1   
3   4          70       RL         60.0     9550   Pave  None      IR1   
4   5          60       RL         84.0    14260   Pave  None      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5       2003   
1       Norm     1Fam     1Story            6            8       1976   
2       Norm     1Fam     2Story            7            5       2001   
3       Norm     1Fam     2Story            7            5       1915   
4       Norm     1Fam     2Story            8            5       2000   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1          1976     Gable  CompShg     MetalSd     MetalSd       None   
2          2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3          1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4          2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0        None   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1          Gd            

**Find Remaining NA Values**

In [195]:
A = trainClean.isna().sum(axis=0)
A[A>0]

LotFrontage    259
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64

Using Previous Logic:  
+ LotFrontage NAs will be converted to the avaerage ration of LotFrontage and  LotArea
+ Convert MasVnrType and MasVnrArea Nas to None and 0 respectively  

The one NA with an unknown eletrical system will be need to be investigated.

In [196]:
trainClean.MasVnrType.fillna('None', inplace = True)
trainClean.MasVnrArea.fillna(0, inplace = True)


In [197]:
trainClean.loc[trainClean['LotFrontage'].isna(),'LotFrontage'] = trainClean['LotArea'] * fRatio

**lets look at the distribution of the values in the electrical column**

In [198]:
trainClean.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

**Very lopsided distribution, I feel asigning the NA to the most common value is the right choice**  
The slight possibility of  introduced error from the value change IMO outweighs the loss of a record in the train data

In [199]:
trainClean.loc[trainClean['Electrical'].isna(), 'Electrical'] = 'SBrkr'    

In [200]:
A = trainClean.isna().sum(axis=0)
A[A>0]

Series([], dtype: int64)

**No More NAs**.....whew!
## Write to new CSV

In [201]:
trainClean.to_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/trainZeroNAs.csv')